Filter we are selecting the required rows from a dataframe syntax: datafram.filter("")

In [0]:
from pyspark.sql.types import *

staticlist = [("    James","Smith","USA","",3000),
    ("    Michael","Rose","USA","NY",2500),
    ("Robert","Williams","USA","CA",6000),
    ("mARIA","Jones           ","USA","FL",20000),
    ("james","Anderson","UK","LND",8000),
    ("MICHEAL","Bevon","UK","LND",3500),
    ("Robert","Patrick","UK","MCR",2800),
    ("Maria","Gonzales","UK","MCR",7000)   
  ]
StructSchema = StructType([
    StructField("firstname", StringType(), True),
    StructField("lastname", StringType(), False),
    StructField("country", StringType(), True),
    StructField("city", StringType(), False),
    StructField("salary", IntegerType(), False)
])

df = spark.createDataFrame(
    data=staticlist,
    schema=StructSchema
)
display(df)

firstname,lastname,country,city,salary
James,Smith,USA,,3000
Michael,Rose,USA,NY,2500
Robert,Williams,USA,CA,6000
mARIA,Jones,USA,FL,20000
james,Anderson,UK,LND,8000
MICHEAL,Bevon,UK,LND,3500
Robert,Patrick,UK,MCR,2800
Maria,Gonzales,UK,MCR,7000


In [0]:
#place a filter where salary is above 5000 (single column filter)
df_filter1 = df.filter("salary > 5000")
df_filter1.show()

+---------+----------------+-------+----+------+
|firstname|        lastname|country|city|salary|
+---------+----------------+-------+----+------+
|   Robert|        Williams|    USA|  CA|  6000|
|    mARIA|Jones           |    USA|  FL| 20000|
|    james|        Anderson|     UK| LND|  8000|
|    Maria|        Gonzales|     UK| MCR|  7000|
+---------+----------------+-------+----+------+



In [0]:
# Multiple Colum filters
#get employees with salary > 5000 and country is USA

df_filter2 = df.filter("salary > 5000 AND country = 'USA'")
df_filter2.show()

+---------+----------------+-------+----+------+
|firstname|        lastname|country|city|salary|
+---------+----------------+-------+----+------+
|   Robert|        Williams|    USA|  CA|  6000|
|    mARIA|Jones           |    USA|  FL| 20000|
+---------+----------------+-------+----+------+



In [0]:
#option 2 # using the datyaframe method

df_filter2 = df.filter(df.salary > 5000).filter(df.country == 'USA')
df_filter2.show()

+---------+----------------+-------+----+------+
|firstname|        lastname|country|city|salary|
+---------+----------------+-------+----+------+
|   Robert|        Williams|    USA|  CA|  6000|
|    mARIA|Jones           |    USA|  FL| 20000|
+---------+----------------+-------+----+------+



In [0]:
# when using datafram AND will be replaced with "&" symbol, OR will be replace with "|" symbal and "=" will be replaced with "==" symbal for not equal "!="

df_filter2 = df.filter ((df.salary > 5000) & (df.country == 'USA'))
df_filter2.show()

+---------+----------------+-------+----+------+
|firstname|        lastname|country|city|salary|
+---------+----------------+-------+----+------+
|   Robert|        Williams|    USA|  CA|  6000|
|    mARIA|Jones           |    USA|  FL| 20000|
+---------+----------------+-------+----+------+



In [0]:
#Multipls columns and Multiple conditions
# If country is usa Then salary > 5000, if country is not usa then salary > 3000

df_filter3 = df.filter(
                        (df.country == 'USA') & (df.salary > 5000) 
                        | 
                        (df.country != 'USA') & (df.salary > 3000)
                      )
df_filter3.show()

+---------+----------------+-------+----+------+
|firstname|        lastname|country|city|salary|
+---------+----------------+-------+----+------+
|   Robert|        Williams|    USA|  CA|  6000|
|    mARIA|Jones           |    USA|  FL| 20000|
|    james|        Anderson|     UK| LND|  8000|
|  MICHEAL|           Bevon|     UK| LND|  3500|
|    Maria|        Gonzales|     UK| MCR|  7000|
+---------+----------------+-------+----+------+



String Functions

In [0]:
#Get the up[per case, lower case and title case for any string
# you can apply these functions inside the withcolumnmethod to apply the required transformations

# get the upper case for above dataframe

from pyspark.sql.functions import *

df_2 = df.withColumn("firstname_upper", upper(df.firstname))\
          .withColumn("lastname_upper", upper(df.lastname))\
          .withColumn("firstname_lower", lower(df.firstname))\
          .withColumn("lastname_lower", lower(df.lastname))\
          .withColumn("firstname_title", initcap(df.firstname))\
          .withColumn("lastname_title", initcap(df.lastname))
df_2.show() 

+-----------+----------------+-------+----+------+---------------+----------------+---------------+----------------+---------------+----------------+
|  firstname|        lastname|country|city|salary|firstname_upper|  lastname_upper|firstname_lower|  lastname_lower|firstname_title|  lastname_title|
+-----------+----------------+-------+----+------+---------------+----------------+---------------+----------------+---------------+----------------+
|      James|           Smith|    USA|    |  3000|          JAMES|           SMITH|          james|           smith|          James|           Smith|
|    Michael|            Rose|    USA|  NY|  2500|        MICHAEL|            ROSE|        michael|            rose|        Michael|            Rose|
|     Robert|        Williams|    USA|  CA|  6000|         ROBERT|        WILLIAMS|         robert|        williams|         Robert|        Williams|
|      mARIA|Jones           |    USA|  FL| 20000|          MARIA|JONES           |          maria|j

In [0]:
#concat firstname anmd last name to create a full name
#concat ("col1", "col2", "col3", "col4"....)
df_2 = df.withColumn("Full_Name",concat(initcap(df.firstname), lit(" "), initcap(df.lastname)))
df_2.show()
    

+-----------+----------------+-------+----+------+--------------------+
|  firstname|        lastname|country|city|salary|           Full_Name|
+-----------+----------------+-------+----+------+--------------------+
|      James|           Smith|    USA|    |  3000|         James Smith|
|    Michael|            Rose|    USA|  NY|  2500|        Michael Rose|
|     Robert|        Williams|    USA|  CA|  6000|     Robert Williams|
|      mARIA|Jones           |    USA|  FL| 20000|Maria Jones      ...|
|      james|        Anderson|     UK| LND|  8000|      James Anderson|
|    MICHEAL|           Bevon|     UK| LND|  3500|       Micheal Bevon|
|     Robert|         Patrick|     UK| MCR|  2800|      Robert Patrick|
|      Maria|        Gonzales|     UK| MCR|  7000|      Maria Gonzales|
+-----------+----------------+-------+----+------+--------------------+



In [0]:
#remove leading and trailing spaces
#TRIM funcation will remove leading and trailing spaces from any  srting
df_trim = df.withColumn("Trimmed_Full_Name", trim(concat(initcap(df.firstname), lit(" "), initcap(df.lastname))))
df_trim.show()

+-----------+----------------+-------+----+------+-----------------+
|  firstname|        lastname|country|city|salary|Trimmed_Full_Name|
+-----------+----------------+-------+----+------+-----------------+
|      James|           Smith|    USA|    |  3000|      James Smith|
|    Michael|            Rose|    USA|  NY|  2500|     Michael Rose|
|     Robert|        Williams|    USA|  CA|  6000|  Robert Williams|
|      mARIA|Jones           |    USA|  FL| 20000|      Maria Jones|
|      james|        Anderson|     UK| LND|  8000|   James Anderson|
|    MICHEAL|           Bevon|     UK| LND|  3500|    Micheal Bevon|
|     Robert|         Patrick|     UK| MCR|  2800|   Robert Patrick|
|      Maria|        Gonzales|     UK| MCR|  7000|   Maria Gonzales|
+-----------+----------------+-------+----+------+-----------------+



In [0]:
#get the left, right and middle values of any given string
# synatx: substring("<colname"),<starting position>,<length of string>

df_ss = df.withColumn("Full_Name",trim(concat(initcap("firstname"),lit(" "),initcap("lastname"))))
df_substring = df_ss.withColumn("Fisrt4letters",substring("Full_Name",0,4))\
                    .withColumn("Last4letters",substring("Full_Name",-4,4))\
                    .withColumn("Middle4letters",substring("Full_Name",3,4))\
                .withColumn("3rddigitrest",substring("Full_Name",3,length("Full_Name")))
df_substring.show()


+-----------+----------------+-------+----+------+---------------+-------------+------------+--------------+-------------+
|  firstname|        lastname|country|city|salary|      Full_Name|Fisrt4letters|Last4letters|Middle4letters| 3rddigitrest|
+-----------+----------------+-------+----+------+---------------+-------------+------------+--------------+-------------+
|      James|           Smith|    USA|    |  3000|    James Smith|         Jame|        mith|          mes |    mes Smith|
|    Michael|            Rose|    USA|  NY|  2500|   Michael Rose|         Mich|        Rose|          chae|   chael Rose|
|     Robert|        Williams|    USA|  CA|  6000|Robert Williams|         Robe|        iams|          bert|bert Williams|
|      mARIA|Jones           |    USA|  FL| 20000|    Maria Jones|         Mari|        ones|          ria |    ria Jones|
|      james|        Anderson|     UK| LND|  8000| James Anderson|         Jame|        rson|          mes | mes Anderson|
|    MICHEAL|   

In [0]:
#get the length of string
#len("colname")
df_len = df_ss.select(df_ss['*'],length("Full_Name").alias("Length"))
df_len.show()
df_len.printSchema()


+-----------+----------------+-------+----+------+---------------+------+
|  firstname|        lastname|country|city|salary|      Full_Name|Length|
+-----------+----------------+-------+----+------+---------------+------+
|      James|           Smith|    USA|    |  3000|    James Smith|    11|
|    Michael|            Rose|    USA|  NY|  2500|   Michael Rose|    12|
|     Robert|        Williams|    USA|  CA|  6000|Robert Williams|    15|
|      mARIA|Jones           |    USA|  FL| 20000|    Maria Jones|    11|
|      james|        Anderson|     UK| LND|  8000| James Anderson|    14|
|    MICHEAL|           Bevon|     UK| LND|  3500|  Micheal Bevon|    13|
|     Robert|         Patrick|     UK| MCR|  2800| Robert Patrick|    14|
|      Maria|        Gonzales|     UK| MCR|  7000| Maria Gonzales|    14|
+-----------+----------------+-------+----+------+---------------+------+

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = false)
 |-- country: string (nu

In [0]:
#check if the column is null
df_null =df.withColumn("isNull",isnull("city"))    
df_null.show()


+-----------+----------------+-------+----+------+------+
|  firstname|        lastname|country|city|salary|isNull|
+-----------+----------------+-------+----+------+------+
|      James|           Smith|    USA|    |  3000| false|
|    Michael|            Rose|    USA|  NY|  2500| false|
|     Robert|        Williams|    USA|  CA|  6000| false|
|      mARIA|Jones           |    USA|  FL| 20000| false|
|      james|        Anderson|     UK| LND|  8000| false|
|    MICHEAL|           Bevon|     UK| LND|  3500| false|
|     Robert|         Patrick|     UK| MCR|  2800| false|
|      Maria|        Gonzales|     UK| MCR|  7000| false|
+-----------+----------------+-------+----+------+------+



In [0]:
list1 = [("Ram","kumar",None),(None,"kumar","Ram"),(None,None,"Ram")]
df_l1 = spark.createDataFrame(data = list1, schema =["Col1","Col2","Col3"])
df_7 = df_l1.withColumn("Coalesce",coalesce("col1","col2","col3"))
df_l1.show()
df_7.show()

+----+-----+----+
|Col1| Col2|Col3|
+----+-----+----+
| Ram|kumar|NULL|
|NULL|kumar| Ram|
|NULL| NULL| Ram|
+----+-----+----+

+----+-----+----+--------+
|Col1| Col2|Col3|Coalesce|
+----+-----+----+--------+
| Ram|kumar|NULL|     Ram|
|NULL|kumar| Ram|   kumar|
|NULL| NULL| Ram|     Ram|
+----+-----+----+--------+



In [0]:
df3 = df_ss.withColumn("split",split("Full_Name"," ",0))
df3.show()
df4 = df3.withColumn("First_split",df3.split[0])\
        .withColumn("Second_split",df3.split[1])
df4.show()


+-----------+----------------+-------+----+------+---------------+------------------+
|  firstname|        lastname|country|city|salary|      Full_Name|             split|
+-----------+----------------+-------+----+------+---------------+------------------+
|      James|           Smith|    USA|    |  3000|    James Smith|    [James, Smith]|
|    Michael|            Rose|    USA|  NY|  2500|   Michael Rose|   [Michael, Rose]|
|     Robert|        Williams|    USA|  CA|  6000|Robert Williams|[Robert, Williams]|
|      mARIA|Jones           |    USA|  FL| 20000|    Maria Jones|    [Maria, Jones]|
|      james|        Anderson|     UK| LND|  8000| James Anderson| [James, Anderson]|
|    MICHEAL|           Bevon|     UK| LND|  3500|  Micheal Bevon|  [Micheal, Bevon]|
|     Robert|         Patrick|     UK| MCR|  2800| Robert Patrick| [Robert, Patrick]|
|      Maria|        Gonzales|     UK| MCR|  7000| Maria Gonzales| [Maria, Gonzales]|
+-----------+----------------+-------+----+------+----

In [0]:
#find the position of a string
#instr function is case sensitive
#instr(<COLNAME>,"substring")
df8 = df.withColumn("position",instr("firstname","J"))
df8.show()

+-----------+----------------+-------+----+------+--------+
|  firstname|        lastname|country|city|salary|position|
+-----------+----------------+-------+----+------+--------+
|      James|           Smith|    USA|    |  3000|       5|
|    Michael|            Rose|    USA|  NY|  2500|       0|
|     Robert|        Williams|    USA|  CA|  6000|       0|
|      mARIA|Jones           |    USA|  FL| 20000|       0|
|      james|        Anderson|     UK| LND|  8000|       0|
|    MICHEAL|           Bevon|     UK| LND|  3500|       0|
|     Robert|         Patrick|     UK| MCR|  2800|       0|
|      Maria|        Gonzales|     UK| MCR|  7000|       0|
+-----------+----------------+-------+----+------+--------+

